In [ ]:
import numpy as np
import csv
import igl
import math
import matplotlib.pyplot as plt
import mpl_toolkits.mplot3d
from scipy.io import loadmat 
import os
import geopandas as gpd
import plotly.graph_objects as go

In [ ]:
def readmitodata(fname):
    """
    Expect a filename of the form "mito_data???.mat" 
    where ??? is the mito number (left padded by 0 is necessary).
    
    argument:
    fname -- string, filename of the mito data
    
    returns:
    mito_dict -- dictionary, keys are the names of the variables in the matlab file
    The keys are:
    'mito' -- the mitochondrion binary shaoe (np.ndarray, has been padded with zeros)
    'vertices' -- the vertices of the meshed mitochondrion (np.ndarray)  
    'faces' -- the faces of the meshed mitochondrion (np.ndarray)
    'cristae_junctions' -- the vertices of the cristae junctions (np.ndarray) or empty list if none
    'min_coords' -- the minimum coordinates of the mitochondrion (np.ndarray) in the original volume    
    'mito_number' -- the number of the mitochondrion (int) from the cc3d.component_labeling function,
                        this is the same as the number in the filename
    
    """
    mito_dict = loadmat(fname)
    # remove matlab meta data
    del mito_dict['__header__']
    del mito_dict['__version__']
    del mito_dict['__globals__']
    mito_dict['mito_number'] = int(mito_dict['mito_number'])
    return mito_dict

In [ ]:
def shortest_path(vecs, faces, vs, vt):

    shortest_path = math.inf

    # get the indices and coordinates of the closest vertices to the source and target points
    ds = igl.signed_distance(vs, vecs, faces)
    svec = faces[ds[1], :]
    svec_coords = vecs[svec, :]
    dt = igl.signed_distance(vt, vecs, faces)
    tvec = faces[dt[1], :]
    tvec_coords = vecs[tvec, :]

    # find the shortest path between the source and target points
    dist_vs_vecs = np.linalg.norm(vs-svec_coords, axis=1)
    # print(dist_vs_vecs)
    dist_vt_vecs = np.linalg.norm(vt-tvec_coords, axis=1)
    # print(dist_vt_vecs)
    dist_vecs = igl.exact_geodesic(vecs, faces, svec, tvec)
    dist = dist_vs_vecs + dist_vecs + dist_vt_vecs
    shortest_path = np.min(dist)

    # for i in range(len(svec)):
    #     for j in range(len(tvec)):
    #         dist_vecs = igl.exact_geodesic(vecs, faces, np.array([svec[i]]), np.array([tvec[j]]))
    #         #print(dist_vecs)
    #         dist = dist_vs_vecs[i] + dist_vecs + dist_vt_vecs[j]
    #         if dist < shortest_path:
    #             shortest_path = dist
    
    return shortest_path

def pair_distance_mesh(vecs, faces, samples):
    npts = np.shape(samples)[0]
    dist = []
    for i in range(npts):
        for j in range(i+1, npts):
            # reshape to (1,3) array
            vs = samples[i].reshape(1,3)
            vt = samples[j].reshape(1,3)
            d = shortest_path(vecs, faces, vs, vt)
            dist.append(d)
    return dist

def mesh_area(vecs, faces):
    double_areas = igl.doublearea(vecs, faces)
    surface_area = np.sum(double_areas) / 2.0
    return surface_area

def ripleyK_mesh(vecs, faces, data, radii):
    K = np.zeros_like(radii)
    area = mesh_area(vecs, faces)
    dists = pair_distance_mesh(vecs, faces, data)
    pair_num = len(dists)
    intensity = pair_num / area
    for i in range(len(radii)):
        K[i] = np.sum(dists < radii[i])
    K = K / intensity
    return K

In [ ]:
current_dir = os.getcwd()
folder_path = os.path.join(current_dir, "mito_data")
path = os.path.join(folder_path, "mito_data004.mat")
mito = readmitodata(path)  

In [ ]:
print(mito.keys())
vertices = np.array(mito['vertices'], dtype=np.float64)
faces = np.array(mito['faces'], dtype=np.int32)
cj = np.array(mito['cristae_junction'], dtype=np.float64)

print(cj)
print(vertices.shape)
print(faces.shape)
print(mito['min_coords'])

In [ ]:

# creating mesh with given face colors
poly = mpl_toolkits.mplot3d.art3d.Poly3DCollection(vertices[faces])
poly.set_facecolor('red') 
poly.set_edgecolor('black')
poly.set_linewidth(0.1)
poly.set_alpha(0.2)

fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')  
ax.add_collection3d(poly) 

ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Z')

# plt.savefig('mesh_plot.png', dpi=300)
plt.show()

In [ ]:
points = cj

"""
Compute distances from a set of points P to a triangle mesh (V,F)  
   Inputs:
    P  #P by 3 list of query point positions
    V  #V by 3 list of vertex positions
    Ele  #Ele by (3|2|1) list of (triangle|edge|point) indices
     
   Outputs:
    sqrD  #P list of smallest squared distances
    I  #P list of primitive indices corresponding to smallest distances
    C  #P by 3 list of closest points
"""
if cj != []:
    squared_distance = igl.point_mesh_squared_distance(points, vertices, faces)
    print(squared_distance)